In [20]:
import re
import numpy as np
import pandas as pd
pd.options.mode.copy_on_write = True 
pd.set_option("display.max_columns", None)
 
import plotly.express as px
import plotly.graph_objects as go

from sklearn import preprocessing 

In [21]:
chinese_dataset_1 = pd.read_csv("../Dataset/The_Chinese_thermal_comfort_dataset/Chinese Thermal Comfort Dataset Class I.csv", encoding="ISO-8859-1")
chinese_dataset_1 = chinese_dataset_1.rename(columns=chinese_dataset_1.iloc[0]).iloc[1:].reset_index(drop=True)

chinese_dataset_2 = pd.read_csv("../Dataset/The_Chinese_thermal_comfort_dataset/Chinese Thermal Comfort Dataset Class II.csv", encoding="ISO-8859-1")
chinese_dataset_2 = chinese_dataset_2.rename(columns=chinese_dataset_2.iloc[0]).iloc[1:].reset_index(drop=True)

chinese_dataset_3 = pd.read_csv("../Dataset/The_Chinese_thermal_comfort_dataset/Chinese Thermal Comfort Dataset Class III.csv", encoding="ISO-8859-1")
chinese_dataset_3 = chinese_dataset_3.rename(columns=chinese_dataset_3.iloc[0]).iloc[1:].reset_index(drop=True)

C:\Users\hoang\AppData\Local\Temp\ipykernel_23028\790989923.py:4: DtypeWarning: Columns (0,9,12,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  chinese_dataset_2 = pd.read_csv("../Dataset/The_Chinese_thermal_comfort_dataset/Chinese Thermal Comfort Dataset Class II.csv", encoding="ISO-8859-1")


In [3]:
useful_cols = ['A4.Season', 'A6.Climate Zone', 
               'C1.Sex', 'C3.Heightxcmx', 'C4.Weightxkgx', 
               'D1.TSV', 'D5.Clothing Insulation (clo)', 'D6.Metabolic Rate (met)', 
               'E1.Indoor Air Temperature (x)', 'E2.Indoor Relative Humidity (%)', 'E3.Indoor Air Velocity (m/s)',
               'F4.PMV', 'F5.PPD']

In [4]:
chinese_dataset_1 = chinese_dataset_1[useful_cols].dropna()
chinese_dataset_2 = chinese_dataset_2[useful_cols].dropna()
chinese_dataset_3 = chinese_dataset_3[useful_cols].dropna()

chinese_dataset = pd.concat([chinese_dataset_1, chinese_dataset_2, chinese_dataset_3]).reset_index(drop=True)
chinese_dataset

,A4.Season,A6.Climate Zone,C1.Sex,C3.Heightxcmx,C4.Weightxkgx,D1.TSV,D5.Clothing Insulation (clo),D6.Metabolic Rate (met),E1.Indoor Air Temperature (x),E1.Indoor Air Temperature (x),E1.Indoor Air Temperature (x),E2.Indoor Relative Humidity (%),E2.Indoor Relative Humidity (%),E2.Indoor Relative Humidity (%),E3.Indoor Air Velocity (m/s),E3.Indoor Air Velocity (m/s),E3.Indoor Air Velocity (m/s),F4.PMV,F5.PPD
0,Winter Season,Cold zone,Female,170,62,-1,1.1,1,18.6,21.4,21.5,32,32,32,0,0,0,-0.58,12.01
1,Winter Season,Cold zone,Female,168,80,-1,1.83,0.9,18.6,21.4,21.5,32,32,32,0,0,0,-0.1,5.19
2,Winter Season,Cold zone,Male,170,82,0,1.36,0.9,18.6,21.4,21.5,32,32,32,0,0,0,-0.66,14.14
3,Winter Season,Cold zone,Male,178,65,0,1.58,0.9,18.6,21.4,21.5,32,32,32,0,0,0,-0.36,7.78
4,Winter Season,Cold zone,Male,174,80,0,1.38,0.9,19.5,20.1,20.9,33,33,33,0,0,0,-0.82,19.45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25438,Transition Season,Cold zone,Male,176,67,1,0.99,1,24.1,24.1,24.1,51,51,51,0.08,0.08,0.13,0.27,6.47
25439,Transition Season,Cold zone,Male,173,63,-1,0.84,1,24.8,24.8,24.8,49,49,49,0.08,0.08,0.05,0.23,6.14
25440,Transition Season,Cold zone,Female,163,48,1,1.05,1,24.8,24.8,24.8,49,49,49,0.08,0.08,0.05,0.51,10.49
25441,Transition Season,Cold zone,Male,177,70,-1,0.71,1,24.7,24.7,24.7,49,49,49,0.09,0.09,0.13,-0.04,5.03


In [5]:
duplicated_columns_list = []
list_of_all_columns = list(chinese_dataset.columns)
for column in list_of_all_columns:
    if list_of_all_columns.count(column) > 1 and not column in duplicated_columns_list:
        duplicated_columns_list.append(column)
duplicated_columns_list

['E1.Indoor Air Temperature (x)',
 'E2.Indoor Relative Humidity (%)',
 'E3.Indoor Air Velocity (m/s)']

In [6]:
for column in duplicated_columns_list:
    list_of_all_columns[list_of_all_columns.index(column)] = column + '_1'
    list_of_all_columns[list_of_all_columns.index(column)] = column + '_2'

chinese_dataset.columns = list_of_all_columns

In [7]:
numeric_cols = ['C3.Heightxcmx', 'C4.Weightxkgx', 
                'D1.TSV', 'D5.Clothing Insulation (clo)', 'D6.Metabolic Rate (met)', 
                'E1.Indoor Air Temperature (x)_1', 'E1.Indoor Air Temperature (x)_2', 'E1.Indoor Air Temperature (x)',
                'E2.Indoor Relative Humidity (%)_1', 'E2.Indoor Relative Humidity (%)_2' ,'E2.Indoor Relative Humidity (%)',
                'E3.Indoor Air Velocity (m/s)_1', 'E3.Indoor Air Velocity (m/s)_2', 'E3.Indoor Air Velocity (m/s)',
                'F4.PMV', 'F5.PPD']

In [8]:
chinese_dataset[numeric_cols] = chinese_dataset[numeric_cols].apply(pd.to_numeric, errors='coerce')

chinese_dataset['Indoor Air Temperature'] = chinese_dataset[['E1.Indoor Air Temperature (x)','E1.Indoor Air Temperature (x)_1','E1.Indoor Air Temperature (x)_2']].mean(axis=1)
chinese_dataset['Indoor Relative Humidity'] = chinese_dataset[['E2.Indoor Relative Humidity (%)_1','E2.Indoor Relative Humidity (%)_2','E2.Indoor Relative Humidity (%)']].mean(axis=1)
chinese_dataset['Indoor Air Velocity'] = chinese_dataset[['E3.Indoor Air Velocity (m/s)_1','E3.Indoor Air Velocity (m/s)_2','E3.Indoor Air Velocity (m/s)']].mean(axis=1)

chinese_dataset = chinese_dataset.drop(columns=['E1.Indoor Air Temperature (x)','E1.Indoor Air Temperature (x)_1','E1.Indoor Air Temperature (x)_2',
                              'E2.Indoor Relative Humidity (%)_1','E2.Indoor Relative Humidity (%)_2','E2.Indoor Relative Humidity (%)',
                              'E3.Indoor Air Velocity (m/s)_1','E3.Indoor Air Velocity (m/s)_2','E3.Indoor Air Velocity (m/s)'])
chinese_dataset

,A4.Season,A6.Climate Zone,C1.Sex,C3.Heightxcmx,C4.Weightxkgx,D1.TSV,D5.Clothing Insulation (clo),D6.Metabolic Rate (met),F4.PMV,F5.PPD,Indoor Air Temperature,Indoor Relative Humidity,Indoor Air Velocity
0,Winter Season,Cold zone,Female,170.0,62.0,-1.0,1.10,1.0,-0.58,12.01,20.500000,32.0,0.000000
1,Winter Season,Cold zone,Female,168.0,80.0,-1.0,1.83,0.9,-0.10,5.19,20.500000,32.0,0.000000
2,Winter Season,Cold zone,Male,170.0,82.0,0.0,1.36,0.9,-0.66,14.14,20.500000,32.0,0.000000
3,Winter Season,Cold zone,Male,178.0,65.0,0.0,1.58,0.9,-0.36,7.78,20.500000,32.0,0.000000
4,Winter Season,Cold zone,Male,174.0,80.0,0.0,1.38,0.9,-0.82,19.45,20.166667,33.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
25438,Transition Season,Cold zone,Male,176.0,67.0,1.0,0.99,1.0,0.27,6.47,24.100000,51.0,0.096667
25439,Transition Season,Cold zone,Male,173.0,63.0,-1.0,0.84,1.0,0.23,6.14,24.800000,49.0,0.070000
25440,Transition Season,Cold zone,Female,163.0,48.0,1.0,1.05,1.0,0.51,10.49,24.800000,49.0,0.070000
25441,Transition Season,Cold zone,Male,177.0,70.0,-1.0,0.71,1.0,-0.04,5.03,24.700000,49.0,0.103333


In [9]:
chinese_dataset['A6.Climate Zone'].unique()

array(['Cold zone', 'Severe cold zone', 'Hot summer and cold winter zone',
       'Sever cold zone', 'Hot summer and warm winter zone ',
       'Hot summer and cold winter zone ', 'Mild zone'], dtype=object)

In [10]:
chinese_dataset.loc[chinese_dataset['A6.Climate Zone']=="Sever cold zone", 'A6.Climate Zone'] = 'Severe cold zone'
chinese_dataset.loc[chinese_dataset['A6.Climate Zone']=='Hot summer and cold winter zone ', 'A6.Climate Zone'] = 'Hot summer and cold winter zone'

In [11]:
one_hot_cols = ['A4.Season','A6.Climate Zone','C1.Sex']

for col in one_hot_cols:
    one_hot = pd.get_dummies(chinese_dataset[col], dtype=float)
    chinese_dataset = chinese_dataset.drop(col,axis = 1)
    chinese_dataset = chinese_dataset.join(one_hot)

In [12]:
chinese_dataset

,C3.Heightxcmx,C4.Weightxkgx,D1.TSV,D5.Clothing Insulation (clo),D6.Metabolic Rate (met),F4.PMV,F5.PPD,Indoor Air Temperature,Indoor Relative Humidity,Indoor Air Velocity,Summer Season,Transition Season,Winter Season,Cold zone,Hot summer and cold winter zone,Hot summer and warm winter zone,Mild zone,Severe cold zone,Female,Male
0,170.0,62.0,-1.0,1.10,1.0,-0.58,12.01,20.500000,32.0,0.000000,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,168.0,80.0,-1.0,1.83,0.9,-0.10,5.19,20.500000,32.0,0.000000,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,170.0,82.0,0.0,1.36,0.9,-0.66,14.14,20.500000,32.0,0.000000,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
3,178.0,65.0,0.0,1.58,0.9,-0.36,7.78,20.500000,32.0,0.000000,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
4,174.0,80.0,0.0,1.38,0.9,-0.82,19.45,20.166667,33.0,0.000000,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25438,176.0,67.0,1.0,0.99,1.0,0.27,6.47,24.100000,51.0,0.096667,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
25439,173.0,63.0,-1.0,0.84,1.0,0.23,6.14,24.800000,49.0,0.070000,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
25440,163.0,48.0,1.0,1.05,1.0,0.51,10.49,24.800000,49.0,0.070000,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
25441,177.0,70.0,-1.0,0.71,1.0,-0.04,5.03,24.700000,49.0,0.103333,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [13]:
chinese_dataset.columns

Index(['C3.Heightxcmx', 'C4.Weightxkgx', 'D1.TSV',
       'D5.Clothing Insulation (clo)', 'D6.Metabolic Rate (met)', 'F4.PMV',
       'F5.PPD', 'Indoor Air Temperature', 'Indoor Relative Humidity',
       'Indoor Air Velocity', 'Summer Season', 'Transition Season',
       'Winter Season', 'Cold zone', 'Hot summer and cold winter zone',
       'Hot summer and warm winter zone ', 'Mild zone', 'Severe cold zone',
       'Female', 'Male'],
      dtype='object')

In [14]:
chinese_dataset.to_csv("../dataset/clean_data/chinese_dataset_before_scale.csv", index=False)

In [15]:
scaler_cols = chinese_dataset.columns.to_list()
scaler_cols.remove('D1.TSV')

In [17]:
min_max = preprocessing.MinMaxScaler()
chinese_dataset[scaler_cols] = min_max.fit_transform(chinese_dataset[scaler_cols])
chinese_dataset['D1.TSV'] = chinese_dataset['D1.TSV'].round()

In [18]:
chinese_dataset['D1.TSV'].unique()

array([-1.,  0., -2.,  1.,  3.,  2., -3.])

In [19]:
chinese_dataset.to_csv('../dataset/clean_data/chinese_data.csv', index=False)